# MQ-7

## Specifications

### Standard Work Condition

    Vc    (Circuit Voltage)        :  5v +/- 0.1
    VH(h) (Heating Voltage - high) :  5v +/- 0.1
    VH(l) (Heating Voltage - low)  :  1.4v +/- 0.1
    RL    (Load Resistance)        :  can adjust
    RH    (Heater Resistance)      :  33 Ohm +/- 5%
    TH(H) (Heating time (high)     :  60 +/- 1 seconds
    TH(L) (Heating time (low)      :  90 +/- 1 seconds
    PH    (Heating consumption)    :  about 350 mW
    

### Sensitivity Characteristic:

    Rs (Sensing Resistance)    : 2kOhm  -  20kOhm (100ppm Carbon Monoxide)
    Preheat Time               : No less than 48 hours
    
    Detecting range:
        Carbon Monoxide:  20ppm  - 2000ppm 


In [1]:
#%pip install numpy

import numpy as np
import math

In [2]:
## Gas concentration values referenced in the datasheet

# Carbon Monoxide
co = {
    'p1' : {'x': 50,   'y':1.605},
    'p2' : {'x': 100,  'y':1.000},
    'p3' : {'x': 400,  'y':0.381},
    'p4' : {'x': 1000, 'y':0.217},
    'p5' : {'x': 4000, 'y':0.090}
}

# Hydrogen
h2 = {
    'p1' : {'x': 50,   'y':1.278},
    'p2' : {'x': 100,  'y':0.792},
    'p3' : {'x': 400,  'y':0.276},
    'p4' : {'x': 1000, 'y':0.141},
    'p5' : {'x': 4000, 'y':0.052}
}

# Liquefied Petroleum Gas
lpg ={
    'p1' : {'x': 50,   'y':9.005},
    'p2' : {'x': 100,  'y':7.989},
    'p3' : {'x': 400,  'y':6.675},
    'p4' : {'x': 1000, 'y':5.993},
    'p5' : {'x': 4000, 'y':4.948}
}

# Methane
ch4 ={
    'p1' : {'x': 50,   'y':13.859},
    'p2' : {'x': 100,  'y':12.745},
    'p3' : {'x': 400,  'y':11.720},
    'p4' : {'x': 1000, 'y':11.039},
    'p5' : {'x': 4000, 'y':9.005}
}

alcohol = {
    'p1' : {'x': 50,   'y':16.194},
    'p2' : {'x': 100,  'y':14.715},
    'p3' : {'x': 400,  'y':13.054},
    'p4' : {'x': 1000, 'y':12.745},
    'p5' : {'x': 4000, 'y':12.004}
}

AIR = 25.836

In [3]:
# RS value calculation
Rs    = lambda Vcc, Vrl, RL : ((Vcc * RL) / Vrl) - RL

# R0 value calculation
R0    = lambda Rs, air : Rs / air

# RS/Ro value calculation (ratio)
Ratio = lambda Rs, R0: Rs / R0

# log(y) = m.log(x) + b
# m
M     = lambda p1, p2 : (math.log10( p2['y'] / p1['y'] )) / (math.log10(p2['x']/p1['x']))
# b
B     = lambda p, m : math.log10(p['y']) - (m * math.log10(p['x']))

# PPM value calculation 
PPM   = lambda  y, m, b: 10 ** ((math.log10(y) - b) / m)

In [4]:
## Calculate the points to be used for the calculation of _m_ and _b_ according to the _ratio_ value 
def getPoints(racio, values):
    size = len(values)
    for idx in range(size - 1):
        if racio >= values[idx]['y']:
            if idx == 0:
                return (values[0], values[1])
            return(values[idx - 1], values[idx])
    return(values[size - 2], values[size - 1])

In [5]:
## Test the _getPoints_ function
"""
points = ['p1', 'p2', 'p3', 'p4', 'p5']
_co  = [ co[n]      for n in list(co)      if n in points]


racio  = 1
p1, p2  = getPoints(racio, _co)

m = M(p1,p2)
b = B(p1, m)

ppm  = PPM(racio, m, b)
print(f"{p1}\n{p2}")
print(f"[ m: {m:.2f} || b: {b:.2f} ]")
print(f"{ppm:.0f}ppm")
"""
print()

In [6]:
points = ['p1', 'p2', 'p3', 'p4', 'p5']

_co      = [ co[n]      for n in list(co)      if n in points]
_h2      = [ h2[n]      for n in list(h2)      if n in points]
_lpg     = [ lpg[n]     for n in list(lpg)     if n in points]
_ch4     = [ ch4[n]     for n in list(ch4)     if n in points]
_alcohol = [ alcohol[n] for n in list(alcohol) if n in points]

#print(_h2)
#print(_lpg)
#print(_ch4)
#print(_co)
#print(_alcohol)

In [7]:
_ratios   = [100, 90, 80, 70, 60, 50, 40, 30, 20, 10,
             9, 8, 7, 6, 5, 4, 3, 2, 1, 
             0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1,
            0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]

h2_ppm      = []
lpg_ppm     = []
ch4_ppm     = []
co_ppm      = []
alcohol_ppm = []

for r in _ratios:
    p1, p2  = getPoints(r, _h2)
    m = M(p1,p2)
    b = B(p1, m)
    h2_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _lpg)
    m = M(p1,p2)
    b = B(p1, m)
    lpg_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _ch4)
    m = M(p1,p2)
    b = B(p1, m)
    ch4_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _co)
    m = M(p1,p2)
    b = B(p1, m)
    co_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _alcohol)
    m = M(p1,p2)
    b = B(p1, m)
    alcohol_ppm.append(PPM(r, m, b))
    


#print(f"{[f'{str(i).center(6)}' for i in _ratio]}")
    
print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Ratio','CO', 'H2', 'LPG', 'CH4', 'Alcohol'))
for i in range(len(_ratios)):
    if _ratios[i] > 5:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(co_ppm[i],3),
                       round(h2_ppm[i],3),
                       round(lpg_ppm[i],3),
                       round(ch4_ppm[i],3),
                       round(alcohol_ppm[i],3)
                      ))
    elif _ratios[i] > 1:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(co_ppm[i],3),
                       round(h2_ppm[i],3),
                       round(lpg_ppm[i],3),
                       round(ch4_ppm[i],3),
                       "----"
                      ))
    else:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(co_ppm[i],3),
                       round(h2_ppm[i],3),
                       "----",
                       "----",
                       "----"
                      ))

Ratio    CO            H2            LPG           CH4           Alcohol      
100      0.117         0.09          0.0           0.0           0.0          
90       0.137         0.105         0.0           0.0           0.0          
80       0.163         0.125         0.0           0.0           0.0          
70       0.198         0.152         0.0           0.0           0.001        
60       0.248         0.189         0.001         0.0           0.004        
50       0.324         0.247         0.002         0.001         0.014        
40       0.45          0.341         0.009         0.008         0.072        
30       0.685         0.517         0.047         0.084         0.577        
20       1.241         0.93          0.493         2.406         10.851       
10       3.427         2.539         27.254        1959.867      274136.289   
9        3.999         2.958         50.161        4015.151      3140290.012  
8        4.753         3.508         99.206        8

In [8]:
# MQ 7 - sensor 1
sensor1 = {}

sensor1['Vcc'] = 5      #volts
sensor1['Vrl'] = 0.15   #volts
sensor1['RL']  = 10     #kOhms
sensor1['AIR'] = AIR   

sensor1['rs'] = Rs(sensor1['Vcc'], sensor1['Vrl'], sensor1['RL'])
sensor1['ro'] = R0(sensor1['rs'], sensor1['AIR'])
sensor1['ratio'] = Ratio(sensor1['rs'], sensor1['ro'])

# MQ 7 - sensor 2
sensor2 = {}

sensor2['Vcc'] = 5      #volts
sensor2['Vrl'] = 0.32   #volts
sensor2['RL']  = 10     #kOhms
sensor2['AIR'] = AIR   

sensor2['rs'] = Rs(sensor2['Vcc'], sensor2['Vrl'], sensor2['RL'])
sensor2['ro'] = R0(sensor2['rs'], sensor2['AIR'])
sensor2['ratio'] = Ratio(sensor2['rs'], sensor2['ro'])



print ("{:<8} {:<8} {:<8} {:<8}".format('Sensor','Rs', 'Ro','Ratio'))
print ("{:<8} {:<8} {:<8} {:<8}".format(1, round(sensor1['rs'],2), round(sensor1['ro'], 2), round(sensor1['ratio'], 2)))
print ("{:<8} {:<8} {:<8} {:<8}".format(2, round(sensor2['rs'],2), round(sensor2['ro'], 2), round(sensor2['ratio'], 2)))
    

Sensor   Rs       Ro       Ratio   
1        323.33   12.51    25.84   
2        146.25   5.66     25.84   


In [9]:
## PPM
sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])

sensor1['h2'] = {}
sensor1['h2']['p1'], sensor1['h2']['p2'] = getPoints(sensor1['ratio'], _h2)
sensor1['h2']['m'] = M(sensor1['h2']['p1'], sensor1['h2']['p2'])
sensor1['h2']['b'] = B(sensor1['h2']['p1'], sensor1['h2']['m'])

sensor1['lpg'] = {}
sensor1['lpg']['p1'], sensor1['lpg']['p2'] = getPoints(sensor1['ratio'], _lpg)
sensor1['lpg']['m'] = M(sensor1['lpg']['p1'], sensor1['lpg']['p2'])
sensor1['lpg']['b'] = B(sensor1['lpg']['p1'], sensor1['lpg']['m'])

sensor1['ch4'] = {}
sensor1['ch4']['p1'], sensor1['ch4']['p2'] = getPoints(sensor1['ratio'], _ch4)
sensor1['ch4']['m'] = M(sensor1['ch4']['p1'], sensor1['ch4']['p2'])
sensor1['ch4']['b'] = B(sensor1['ch4']['p1'], sensor1['ch4']['m'])

sensor1['alcohol'] = {}
sensor1['alcohol']['p1'], sensor1['alcohol']['p2'] = getPoints(sensor1['ratio'], _alcohol)
sensor1['alcohol']['m'] = M(sensor1['alcohol']['p1'], sensor1['alcohol']['p2'])
sensor1['alcohol']['b'] = B(sensor1['alcohol']['p1'], sensor1['alcohol']['m'])


####
sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])

sensor2['h2'] = {}
sensor2['h2']['p1'], sensor2['h2']['p2'] = getPoints(sensor2['ratio'], _h2)
sensor2['h2']['m'] = M(sensor2['h2']['p1'], sensor2['h2']['p2'])
sensor2['h2']['b'] = B(sensor2['h2']['p1'], sensor2['h2']['m'])

sensor2['lpg'] = {}
sensor2['lpg']['p1'], sensor2['lpg']['p2'] = getPoints(sensor2['ratio'], _lpg)
sensor2['lpg']['m'] = M(sensor2['lpg']['p1'], sensor2['lpg']['p2'])
sensor2['lpg']['b'] = B(sensor2['lpg']['p1'], sensor2['lpg']['m'])

sensor2['ch4'] = {}
sensor2['ch4']['p1'], sensor2['ch4']['p2'] = getPoints(sensor2['ratio'], _ch4)
sensor2['ch4']['m'] = M(sensor2['ch4']['p1'], sensor2['ch4']['p2'])
sensor2['ch4']['b'] = B(sensor2['ch4']['p1'], sensor2['ch4']['m'])

sensor2['alcohol'] = {}
sensor2['alcohol']['p1'], sensor2['alcohol']['p2'] = getPoints(sensor2['ratio'], _alcohol)
sensor2['alcohol']['m'] = M(sensor2['alcohol']['p1'], sensor2['alcohol']['p2'])
sensor2['alcohol']['b'] = B(sensor2['alcohol']['p1'], sensor2['alcohol']['m'])


####

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio', 'CO', 'H2', 'LPG', 'CH4', 'Alcohol'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(PPM(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['h2']['m'],      sensor1['h2']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['lpg']['m'],     sensor1['lpg']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['ch4']['m'],     sensor1['ch4']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['alcohol']['m'], sensor1['alcohol']['b']),2),
          ))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(PPM(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['h2']['m'],      sensor2['h2']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['lpg']['m'],     sensor2['lpg']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['ch4']['m'],     sensor2['ch4']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['alcohol']['m'], sensor2['alcohol']['b']),2),
          ))

Volt     Ratio    CO            H2            LPG           CH4           Alcohol      
0.15     25.84    0.85          0.64          0.11          0.29          1.7          
0.32     25.84    0.85          0.64          0.11          0.29          1.7          


In [10]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor1['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor1['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio', 'CO', 'H2', 'LPG', 'CH4','Alcohol'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    
    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])

    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])

    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    if r >4:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    elif r > 1:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   "----",
                   "----",
                   "----",
                  ))

    

    Volt     Ratio    CO            H2            LPG           CH4           Alcohol      
0   0.15     25.84    0.85          0.64          0.11          0.29          1.7          
1   0.25     15.18    1.86          1.39          2.43          23.52         79.76        
2   0.35     10.62    3.14          2.33          19.28         1304.72       68732.12     
3   0.45     8.08     4.68          3.46          93.7          8370.32       38167492.97  
4   0.55     6.47     6.49          4.78          524.88        38166.22      6638221898.22
5   0.65     5.35     8.57          6.29          2280.8        138908.63     536535531155.41
6   0.75     4.53     10.94         8.0           7599.86       431051.79     25217310497898.73
7   0.85     3.9      13.61         9.93          22330.64      1188332.4     ----         
8   0.95     3.41     16.6          12.08         59570.07      2991301.89    ----         
9   1.05     3.01     19.94         14.48         147251.98     7008838.03

In [11]:
## Projection - Increasing VRL

# Sensor 2

Vcc    = 5 #volts
vrl    = np.arange(sensor2['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor2['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio', 'CO', 'H2', 'LPG', 'CH4','Alcohol'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    
    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])

    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])

    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    if r >4:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    elif r > 1:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   "----",
                   "----",
                   "----",
                  ))

    

    Volt     Ratio    CO            H2            LPG           CH4           Alcohol      
0   0.32     25.84    0.85          0.64          0.11          0.29          1.7          
1   0.42     19.26    1.31          0.98          0.61          3.28          14.23        
2   0.52     15.22    1.85          1.38          2.4           23.04         78.35        
3   0.62     12.48    2.48          1.84          7.56          141.56        1626.51      
4   0.72     10.5     3.19          2.37          20.53         1404.88       88384.51     
5   0.82     9.01     4.0           2.96          49.99         3999.49       3098842.58   
6   0.92     7.83     4.9           3.62          116.28        10322.06      77834383.73  
7   1.02     6.89     5.91          4.35          312.14        24669.97      1505648182.8 
8   1.12     6.12     7.04          5.17          836.84        55447.94      23634489438.92
9   1.22     5.47     8.29          6.08          1927.24       118550.93     3

In [12]:
## projectionRo Ro - Fixing VRL and Varying the Ro

# Sensor 1


Vcc    = 5 #volts
Vrl    = sensor1['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor1['rs']   
projectionRo['ro'] = np.arange(sensor1['ro'] - sensor1['ro']/10 , sensor1['ro'] + sensor1['ro']/10 , 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Rs', 'Ro', 'Ratio', 'CO', 'H2', 'LPG', 'CH4','Alcohol'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    
    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])

    projectionRo['h2'] = {}
    projectionRo['h2']['p1'], projectionRo['h2']['p2'] = getPoints(r, _h2)
    projectionRo['h2']['m'] = M(projectionRo['h2']['p1'], projectionRo['h2']['p2'])
    projectionRo['h2']['b'] = B(projectionRo['h2']['p1'], projectionRo['h2']['m'])
    
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])

    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])

    projectionRo['alcohol'] = {}
    projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'] = getPoints(r, _alcohol)
    projectionRo['alcohol']['m'] = M(projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'])
    projectionRo['alcohol']['b'] = B(projectionRo['alcohol']['p1'], projectionRo['alcohol']['m'])

    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['h2']['m'], projectionRo['h2']['b']),2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
               round(PPM(r, projectionRo['alcohol']['m'], projectionRo['alcohol']['b']),2),
              ))


    Volt     Rs       Ro       Ratio    CO            H2            LPG           CH4           Alcohol      
0   0.15     323.33   11.26    28.71    0.73          0.55          0.06          0.12          0.79         
1   0.15     323.33   11.36    28.45    0.74          0.56          0.06          0.13          0.85         
2   0.15     323.33   11.46    28.21    0.75          0.57          0.07          0.14          0.9          
3   0.15     323.33   11.56    27.96    0.76          0.57          0.07          0.15          0.96         
4   0.15     323.33   11.66    27.72    0.77          0.58          0.07          0.16          1.02         
5   0.15     323.33   11.76    27.49    0.78          0.59          0.08          0.17          1.09         
6   0.15     323.33   11.86    27.25    0.79          0.59          0.08          0.19          1.16         
7   0.15     323.33   11.96    27.03    0.8           0.6           0.09          0.2           1.23         
8   0.15  

In [13]:
## projectionRo Ro - Fixing VRL and Varying the Ro

# Sensor 1


Vcc    = 5 #volts
Vrl    = sensor2['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor2['rs']   
projectionRo['ro'] = np.arange(sensor2['ro'] - sensor2['ro']/10 , sensor2['ro'] + sensor2['ro']/10 , 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt',  'Rs', 'Ro', 'Ratio', 'CO', 'H2', 'LPG', 'CH4','Alcohol'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    
    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])

    projectionRo['h2'] = {}
    projectionRo['h2']['p1'], projectionRo['h2']['p2'] = getPoints(r, _h2)
    projectionRo['h2']['m'] = M(projectionRo['h2']['p1'], projectionRo['h2']['p2'])
    projectionRo['h2']['b'] = B(projectionRo['h2']['p1'], projectionRo['h2']['m'])
    
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])

    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])

    projectionRo['alcohol'] = {}
    projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'] = getPoints(r, _alcohol)
    projectionRo['alcohol']['m'] = M(projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'])
    projectionRo['alcohol']['b'] = B(projectionRo['alcohol']['p1'], projectionRo['alcohol']['m'])

    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['h2']['m'], projectionRo['h2']['b']),2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
               round(PPM(r, projectionRo['alcohol']['m'], projectionRo['alcohol']['b']),2),
              ))


    Volt     Rs       Ro       Ratio    CO            H2            LPG           CH4           Alcohol      
0   0.32     146.25   5.09     28.71    0.73          0.55          0.06          0.12          0.79         
1   0.32     146.25   5.19     28.15    0.75          0.57          0.07          0.14          0.91         
2   0.32     146.25   5.29     27.62    0.77          0.58          0.08          0.17          1.05         
3   0.32     146.25   5.39     27.11    0.8           0.6           0.08          0.19          1.2          
4   0.32     146.25   5.49     26.62    0.82          0.61          0.09          0.23          1.37         
5   0.32     146.25   5.59     26.14    0.84          0.63          0.1           0.26          1.56         
6   0.32     146.25   5.69     25.68    0.86          0.65          0.12          0.3           1.78         
7   0.32     146.25   5.79     25.24    0.88          0.66          0.13          0.35          2.01         
8   0.32  